In [2]:
import pandas as pd
# 챗봇 : 한국어 자연어처리 모델
from sentence_transformers import SentenceTransformer
# 기존데이터와 입력된 데이터의 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
# 한국어 자연어처리 모델을 가져옴. : 구글 BERT
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?","한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)
print(embeddings)
embeddings.shape

[[-0.37510458 -0.7733839   0.5927711  ...  0.57923526  0.3268346
  -0.65089655]
 [-0.09361681 -0.18191518 -0.19230832 ... -0.03165777  0.30412534
  -0.2679363 ]]


(2, 768)

3개 파일 가져오기
1. wellness_dataset_original.csv
2. ChatBotData.csv
3. chat.html

In [4]:
# 데이터 불러오기 - 심리상담 데이터
df = pd.read_csv('wellness_dataset_original.csv')
df.head(3)

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN


In [5]:
len(df)

5231

In [6]:
# 필요없는 컬럼 삭제 - Unnamed: 3 컬럼 삭제
df = df.drop(columns=['Unnamed: 3'])


In [7]:
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [8]:
# nan 데이터가 있으면 모두 행 삭제
# df = df.dropna()
# df.head()

In [9]:
# 챗봇에 NaN 데이터 삭제
# isna : nan데이터 있는 위치 index
# row(행)으로 삭제 : df = df.dropna() / columns(열)로 삭제 : df.dropna(axis=1) 
df = df[~df['챗봇'].isna()]
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [10]:
# 0번째 row의 유저 질문 확인
df.loc[0,'유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [11]:
model.encode(df.loc[0,'유저'])

array([-4.80606616e-01, -2.94869423e-01,  4.37900215e-01, -6.40137851e-01,
        3.28670740e-02, -3.42647374e-01, -5.47481179e-02,  1.73056182e-02,
       -4.08221096e-01, -5.06034076e-01, -1.68733463e-01, -3.98676932e-01,
       -1.24776840e-01, -9.71538201e-02, -1.65286884e-01,  5.72612369e-03,
        6.13690987e-02, -1.91312388e-01,  2.53917605e-01, -5.85019708e-01,
       -2.84425557e-01, -2.32034907e-01, -3.27080548e-01,  6.72975630e-02,
       -1.62805827e-05, -4.72336292e-01, -3.60021859e-01,  2.91879863e-01,
       -6.63861156e-01, -3.10574383e-01,  5.79524994e-01, -3.11722606e-01,
        1.47696752e-02, -2.12172851e-01,  2.22057894e-01, -1.73828974e-01,
       -3.78458500e-01, -4.20398414e-01, -2.38218918e-01,  6.38703927e-02,
       -1.15304396e-01, -2.44563922e-01, -5.00228345e-01,  1.68355241e-01,
       -6.58360124e-01, -8.91941845e-01, -6.26956999e-01, -3.21965545e-01,
       -7.05358803e-01,  3.71447295e-01, -5.45803428e-01,  7.76295438e-02,
        1.09864593e-01,  

In [12]:
# 유저 질문 개수
len(df)

1034

In [13]:
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [16]:
# 1034 1차원 행렬로 변경
df['embedding'] = pd.Series([[]]*len(df)) # 1034
# df['유저']
df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))
df.head(3)

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792..."


In [17]:
# 예제 질문1
text = '요즘 머리가 너무 아프고 너무 힘들어요'
test_em = model.encode(text)

In [18]:
test_em

array([-9.25968885e-01,  1.49269909e-01,  5.89155912e-01, -3.48825544e-01,
       -3.11765194e-01,  2.41537705e-01, -3.68349165e-01,  3.09100091e-01,
        1.28971562e-01, -4.13765937e-01, -5.21825105e-02, -3.09242100e-01,
        3.11593413e-01,  4.81443584e-01, -4.94652659e-01,  1.78202093e-01,
        2.37989515e-01,  4.37562257e-01, -1.59708500e-01, -4.67824250e-01,
       -7.56812021e-02,  3.23594451e-01,  9.55730975e-01,  2.53328308e-02,
        3.23572516e-01, -6.67475462e-01, -6.43570125e-02, -6.02233447e-02,
       -1.31186679e-01,  6.51322842e-01,  4.22071755e-01, -4.88138676e-01,
        7.83526897e-02,  2.08481357e-01,  1.69788107e-01,  3.08408052e-01,
        3.54874551e-01,  1.67552665e-01, -7.32453942e-01,  5.60936593e-02,
       -3.76547545e-01,  1.08094931e+00, -3.53890926e-01, -1.25327811e-01,
       -9.03611183e-01, -4.07017350e-01,  1.15737699e-01,  2.49573126e-01,
       -4.25571561e-01, -5.00578918e-02, -1.02957809e+00, -1.07779548e-01,
       -7.25825951e-02,  

In [19]:
# 정확도 예측 : embedding컬럼의 값과 test_em의 값을 비교해서 거리를 환산한 정확도를 리턴
df['distance'] = df['embedding'].map(lambda x:cosine_similarity([test_em],[x]).squeeze())
df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.29486942, 0.43790022, -0.6401...",0.459298
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.14506274, 0.2949036, -0.673949...",0.490708
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652004, -0.081268355, 1.0945568, 0.105792...",0.365203
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285071, -0.5076045...",0.415008
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277718, -0.37413934, 0.04053212, -0.8624...",0.320734


In [21]:
# 가장 높은 유사도를 출력
# idxmax() : 가장 큰값의 위치값을 리턴
answer = df.loc[df['distance'].idxmax()]
print('질문 : ',text)
print('유사한 질문 : ',answer['유저'])
print('챗봇의 답변 : ',answer['챗봇'])
print('유사도% : ',answer['distance'])

질문 :  요즘 머리가 너무 아프고 너무 힘들어요
유사한 질문 :  요즘은 머리가 한쪽만 지그시 누르는 것처럼 무겁고 아파요.
챗봇의 답변 :  으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
유사도% :  0.8284860849380493
